# 02 - 随机实验


## 黄金标准

在上一节中，我们探讨了相关性与因果性之间的差异及其原因，并明确了将相关转化为因果所需的条件。

$
E[Y|T=1] - E[Y|T=0] = \underbrace{E[Y_1 - Y_0|T=1]}_{ATT} + \underbrace{\{ E[Y_0|T=1] - E[Y_0|T=0] \}}_{BIAS}
$


回顾一下，若不存在偏误（bias），相关即可转化为因果。当 $E[Y_0|T=0]=E[Y_0|T=1]$ 时，偏误便不会产生。换言之，若处理组与对照组除处理因素外其他条件均等或可比，则相关即成为因果。或用更专业的术语来说，当未处理组的结果等同于处理组的反事实结果时。需记住，此反事实结果指的是处理组若未接受处理时将会出现的结果。

我认为我们在用数学术语解释如何将相关等同于因果关系方面做得还算不错。但那仅是理论层面的探讨。现在，我们介绍消除偏误的第一个工具：**随机实验（Randomised Experiments）**。随机实验将群体中的个体随机分配到处理组或控制组。接受处理的比例不必是50%，实验中可能仅有10%的样本接受处理。

随机化通过使潜在结果独立于处理方式，彻底消除了偏误。

$
(Y_0, Y_1) \perp\!\!\!\perp T
$

这一点起初可能令人困惑（对我而言确实如此）。但别担心，我会进一步解释的。如果结果和处理方式无关，这是否也意味着处理毫无效果？确实如此！但请注意，我这里说的不是实际结果（outcomes）而是**潜在结果（potential outcomes）**。潜在结果指的是在接受处理（$Y_1$）或处于对照组（$Y_0$）时可能出现的结果。在随机实验中，我们不希望结果独立于处理方式，因为我们相信处理会导致结果变化。但我们期望潜在结果能独立于处理方式。

![img](./images/02/indep.png)

所谓潜在结果独立于处理，即意味着在期望值上，处理组与对照组的结果是相同的。简而言之，这表明处理组和对照组具有可比性。或者说，了解处理分配情况并不能提供任何关于处理前结果的信息。因此，$(Y_0, Y_1)\perp T$ 意味着：处理是导致处理组与对照组结果差异的唯一因素。要理解这一点，需注意到独立性恰好意味着

 
$
E[Y_0|T=0]=E[Y_0|T=1]=E[Y_0]
$

正如我们所见，这使得

$
E[Y|T=1] - E[Y|T=0] = E[Y_1 - Y_0]=ATE
$

因此，随机化为我们提供了一种方法，即通过处理组与对照组均值的简单差异来定义处理效应。


## 一个遥远学校的例子

2020 年，新冠疫情迫使企业适应社交距离措施。送货服务变得普及，大型企业转向远程工作策略。学校也不例外，许多机构开始建立自己的在线课程资源库。

危机爆发四个月后，许多人思考这些引入的变革能否持续。毫无疑问，在线学习有其优势：它能节省房地产和交通成本，因而更为经济；同时，通过整合全球顶尖教学内容而非局限于固定教师群体，它还能实现更高程度的数字化。尽管如此，我们仍需解答在线学习对学生学业表现究竟产生消极还是积极影响。

一种解法方法是选取主要提供在线课程的学校学生，与采用面对面授课的学校学生进行比较。但如今我们已明白，这并非最佳途径。可能在线学校仅吸引了自律性强、即使面对面授课也能表现优异的学生群体。若如此，我们将面临正向偏误——接受在线教育的学生学术表现优于未接受者： $E[Y_0|T=1] > E[Y_0|T=0]$.

反之，也可能是因为在线课程更便宜，主要吸引经济条件较差、需半工半读的学生群体。在这种情况下，即便他们参与面对面授课，其表现仍可能逊于传统学校学生。若该假设成立，则会出现反向偏误——接受在线教育的学生学术表现反而更差： $E[Y_0|T=1] < E[Y_0|T=0]$. 

因此，尽管可进行简单对比，其结论却缺乏说服力。无论如何，我们永远无法确定是否存在潜在偏误掩盖了真实的因果效应。

![img](./images/02/lurking_bias.png)

为解决这一问题，我们需要使处理组与未处理组具备可比性 $E[Y_0|T=1] = E[Y_0|T=0]$。实现这一目标的方法是随机将在线课程和面对面课程分配给学生。若能成功实施，除所接受的处理方式不同外，处理组和未处理组在其他方面平均而言将保持一致。

幸运的是，已有经济学家为我们完成了这项工作。他们通过随机分配课程，使部分学生接受面对面授课，另一部分仅参与在线课程，第三组则采用线上线下混合教学模式。学期末，研究者收集了标准化考试的成绩数据。

数据呈现如下：

```{dropdown} 查看 Stata 代码
```stata
* Direct import from GitHub URL (Recommended for Stata 16+)
* import delimited "https://raw.githubusercontent.com/matheusfacure/python-causality-handbook/master/causal-inference-for-the-brave-and-true/data/online_classroom.csv", clear

* Clear any existing data from memory
clear

* Import the CSV file from local path
import delimited "./data/online_classroom.csv", encoding(UTF-8) 

* Alternative import command (for Stata versions before 14)
* insheet using "./data/online_classroom.csv", clear

* Check if data loaded correctly
describe  // Display variable names and types
list in 1/5  // Show first 5 observations

In [10]:
import pandas as pd
import numpy as np

data = pd.read_csv("./data/online_classroom.csv")
print(data.shape)
data.head()

(323, 10)


gender  asian  black  hawaiian  hispanic  unknown  white  format_ol  \
0       0    0.0    0.0       0.0       0.0      0.0    1.0          0   
1       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
2       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
3       1    0.0    0.0       0.0       0.0      0.0    1.0          0   
4       1    0.0    0.0       0.0       0.0      0.0    1.0          1   

   format_blended  falsexam  
0             0.0  63.29997  
1             0.0  79.96000  
2             1.0  83.37000  
3             1.0  90.01994  
4             0.0  83.30000

可以看到我们拥有 323 个样本。虽然称不上大数据，但足以开展工作。为了估计因果效应，我们可以简单地计算各处理组的平均得分。

```{dropdown} 查看 Stata 代码
```stata
* Step 1: Create class_format variable
gen class_format = "face_to_face"  // Set default value
replace class_format = "online" if format_ol == 1
replace class_format = "blended" if format_blended == 1

* Step 2: Calculate means by group
collapse (mean) asian black falsexam format_blended format_ol gender, by(class_format)

* Step 3: Display results
list, noobs clean

In [14]:
(data
 .assign(class_format = np.select(
     [data["format_ol"].astype(bool), data["format_blended"].astype(bool)],
     ["online", "blended"],
     default="face_to_face"
 ))
 .groupby(["class_format"])
 .mean())

gender     asian     black  hawaiian  hispanic   unknown  \
class_format                                                               
blended       0.550459  0.217949  0.102564  0.025641  0.012821  0.012821   
face_to_face  0.633333  0.202020  0.070707  0.000000  0.010101  0.000000   
online        0.542553  0.228571  0.028571  0.014286  0.028571  0.000000   

                 white  format_ol  format_blended   falsexam  
class_format                                                  
blended       0.628205        0.0             1.0  77.093731  
face_to_face  0.717172        0.0             0.0  78.547485  
online        0.700000        1.0             0.0  73.635263

是的，就这么简单。我们可以看到，面对面课程的平均得分为 78.54 分，而在线课程的平均得分为 73.63 分。这对在线学习的支持者来说不是什么好消息。因此，在线课程的 $ATE$ 为 -4.91。这意味着，平均而言，**在线课程会导致学生的成绩下降约 5 分**。就是这样。你无需担心在线课程可能有负担不起面对面课程的较差学生，或者就此而言，你不必担心不同处理组的学生在接受处理之外有任何其他差异。通过设计，随机实验旨在消除这些差异。

因此，验证随机化过程是否正确（或确认所查看的数据无误）的一个有效方法是检查处理组与对照组在预处理变量上是否均衡。我们的数据包含性别和种族信息，可用于观察各群体间这些特征是否相似。可以看出，在 `gender` 、 `asian` 、 `hispanic` 和 `white` 变量上，两组表现相当接近。然而， `black` 变量似乎存在些许差异。这提醒我们注意小样本数据集可能出现的情况：即使在随机化条件下，也可能偶然出现组间差异。而在大样本中，此类差异往往会消失。

## 理想实验

随机实验或随机对照试验（RCT），是获得因果效应最可靠的方法。这种方法非常直接，效果也极具说服力，强大到很多国家都要求用它来验证新药是否有效。

打个通俗的比方：你可以把 RCT 想象成高铁，而其他因果推断方法更像是大巴车。大巴也能把人从一个地方送到另一个地方，但路上堵车、颠簸不定、速度慢。而高铁速度快、路线清晰、几乎不受干扰。

所以说，如果条件允许，我们当然希望所有的因果研究都能坐“高铁”。一个设计精良的随机对照试验，是科学家心目中的理想工具。

![img](./images/02/science_dream.png)

遗憾的是，这些方法往往要么成本极高，要么完全违背伦理。有时，我们根本无法控制分配机制。设想你是一名医生，试图评估孕期吸烟对新生儿体重的影响，你不可能随机强制一部分孕妇在孕期吸烟。又或者，假设你供职于一家大型银行，需要评估信用额度对客户流失率的影响，向客户随机分配信用额度的成本将过于高昂。再比如，你想了解提高最低工资对失业率的影响，但显然语法随意指定各国采用不同的最低工资标准。道理不言自明。

后文我们将探讨如何通过条件随机化降低实验成本，但对于违背伦理或不可行的实验仍束手无策。尽管如此，在处理因果问题时，始终值得思考理想实验的形态。不妨自问：倘若可能，为揭示这一因果效应，你会设计怎样的完美实验？这种思考往往能为我们指明方向，即便在缺乏理想实验条件时，也能找到揭示因果效应的途径。


## 分配机制

在随机实验中，将个体分配到处理组或控制组的机制是随机的。正如我们之后会看到的，所有的因果推断方法，其实都在试图识别“处理是如何被分配的”这一机制。如果我们能够明确知道这个机制是如何运作的，即使它不是随机的，我们的因果推断也会更加可信。

但遗憾的是，这种“分配机制”不能仅靠观察数据就识别出来。举个例子：假设你有一个数据集，显示“受教育程度”和“财富”之间存在相关性。你无法仅凭这点判断到底是教育导致了财富，还是财富影响了教育。你必须依赖自己对现实世界的理解，来推断一个合理的分配机制。

比如你可以认为：学校通过教育提升了人的生产力，从而让人获得更高收入；或者如果你对教育持悲观态度，也可以说学校对生产力没什么帮助，这种相关性只是因为富裕家庭更有条件让孩子接受高等教育，所以它是一个“伪相关”。

在因果问题中，我们常常可以提出两种相反的解释：一种是“X 导致了 Y”，另一种是“有个第三变量 Z 同时导致了 X 和 Y，从而造成 X 与 Y 之间的表面相关”。

正因为如此，了解和掌握“处理是如何被分配的”这一机制，会让你的因果结论更有说服力。这也是因果推断最吸引人的地方之一：应用机器学习往往只是“按对顺序点按钮”，而应用因果推断则要求你深入思考数据背后真正的生成机制。


## 核心要点

我们探讨了随机实验是揭示因果影响最简单且最有效的方法。它之所以有效，是因为它让处理组和控制组在其他方面具有可比性。

当然，我们并不能在所有情况下都进行随机实验，但思考一下“如果可以做，我们会怎么设计理想实验”依然是非常有帮助的。

这时候，熟悉统计学的人可能会跳出来质疑：“你没有考虑因果效应估计值的方差啊！怎么知道这 4.91 分的下降不是偶然的呢？”换句话说，怎么判断这个差异是否具有统计显著性呢？

他们的质疑是完全正确的。别担心，我接下来就会回顾一些相关的统计概念。




## 参考文献
我愿将这一系列作品视为对 Joshua Angrist、Alberto Abadie 和 Christopher Walters 杰出计量经济学课程的致敬。第一部分的大部分思想源自他们在美国经济学会授课的内容。在艰难的 2020 年，正是观看他们的课程视频让我保持了理智。

 - [Cross-Section Econometrics](https://www.aeaweb.org/conference/cont-ed/2017-webcasts)
 - [Mastering Mostly Harmless Econometrics](https://www.aeaweb.org/conference/cont-ed/2020-webcasts)

 我还想引用 Angrist 的精彩著作。它们向我展示了计量经济学（他们称之为“Metrics”）不仅极为实用，而且充满乐趣。
 - [Mostly Harmless Econometrics](https://www.mostlyharmlesseconometrics.com)
 - [Mastering ‘Metrics](https://www.masteringmetrics.com)

最后还要感谢 Miguel Hernán 和 Jamie Robins 的[《Causal Inference》](https://hsph.harvard.edu/profile/miguel-hernan/)一书。它是我在面对最棘手的因果问题时的可靠伙伴。

![img](./images/poetry.png)

## 参与贡献

**《Causal Inference for the Brave and True》** 是一本关于因果推断的开源教材，致力于以经济上可负担、认知上可理解的方式，普及这门“科学的统计基础”。全书基于 Python，仅使用自由开源软件编写，原始英文版本由 [Matheus Facure](https://github.com/matheusfacure) 编写与维护。

本书的中文版由黄文喆与许文立助理教授合作翻译，并托管在 [GitHub 中文主页](https://github.com/Wenzhe-Huang/python-causality-handbook-zh)。希望本地化的内容能帮助更多中文读者学习和掌握因果推断方法。

如果你觉得这本书对你有帮助，并希望支持该项目，可以前往 [Patreon](https://www.patreon.com/causal_inference_for_the_brave_and_true) 支持原作者。

如果你暂时不方便进行经济支持，也可以通过以下方式参与贡献：

* 修正错别字
* 提出翻译或表达建议
* 反馈你未能理解的部分内容

欢迎前往英文版或中文版仓库点击 [issues 区](https://github.com/matheusfacure/python-causality-handbook/issues) 或 [中文版 issues 区](https://github.com/Wenzhe-Huang/python-causality-handbook-zh/issues) 提出反馈。

最后，如果你喜欢这本书的内容，也请将其分享给可能感兴趣的朋友，并为项目在 GitHub 上点亮一颗星：[英文版仓库](https://github.com/matheusfacure/python-causality-handbook) / [中文版仓库](https://github.com/Wenzhe-Huang/python-causality-handbook-zh)。